In [1]:
import torch
import timm
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [3]:
# Load dataset
dataset = datasets.ImageFolder(root='Monkeypox Skin Image Dataset', transform=transform)
num_classes = len(dataset.classes)  # Should be 4 for your case
print("num_classes--",num_classes)

num_classes-- 4


In [4]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
# Define CNN model
class MonkeypoxCNN(nn.Module):
    def __init__(self):
        super(MonkeypoxCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)   # (3,224,224) -> (32,224,224)
        self.pool = nn.MaxPool2d(2, 2)                            # -> (32,112,112)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # -> (64,112,112)
        self.pool2 = nn.MaxPool2d(2, 2)                           # -> (64,56,56)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)# -> (128,56,56)
        self.pool3 = nn.MaxPool2d(2, 2)                           # -> (128,28,28)

        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)  # 4 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Instantiate model

In [6]:
# Instantiate model
model = MonkeypoxCNN().to(device)
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

Epoch 1/10, Loss: 1.2605, Accuracy: 48.96%
Epoch 2/10, Loss: 1.0517, Accuracy: 59.09%
Epoch 3/10, Loss: 0.9551, Accuracy: 62.47%
Epoch 4/10, Loss: 0.8495, Accuracy: 65.19%
Epoch 5/10, Loss: 0.7476, Accuracy: 71.95%
Epoch 6/10, Loss: 0.6511, Accuracy: 75.97%
Epoch 7/10, Loss: 0.4986, Accuracy: 81.30%
Epoch 8/10, Loss: 0.3797, Accuracy: 85.58%
Epoch 9/10, Loss: 0.2887, Accuracy: 88.96%
Epoch 10/10, Loss: 0.2300, Accuracy: 93.12%
